<a href="https://colab.research.google.com/github/lavanyaasehgal/IntelliShorts/blob/main/NLP_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [1]:
!pip install Flask -q
!pip install contractions -q
!pip install -U transformers -q
!pip install -U accelerate -q
!pip install keras_nlp -q
!pip install datasets -q
!pip install huggingface-hub -q
!pip install rouge-score -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !pip install gnews -q
# !pip install GoogleNews -q
# !pip install fake_useragent -q
# !pip install newspaper3k -q
# !pip install newsapi-python -q
# !pip install pycountry -q
# !pip install gnewsclient -q

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from string import punctuation
import string, re, nltk
nltk.download("all")
!python -m spacy download en_core_web_sm
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import spacy
import contractions
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import AdamW
from keras.utils import to_categorical
import torch
from transformers import AutoTokenizer
import transformers
from transformers import TFAutoModelForSeq2SeqLM, AutoModelForSeq2SeqLM,DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_metric, load_dataset
import accelerate

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

2023-10-28 16:45:59.336444: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-28 16:45:59.336523: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-28 16:45:59.336573: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-28 16:45:59.351242: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-28 16:46:06.096180: W tensorflow/compiler/

# Cleaning (functions)

In [7]:
#Regular Expression
regexp = RegexpTokenizer("[\w']+")
#Lowercase
def text_lower(text):
  text = text.lower()
  return text
#Remove Whitespace
def remove_whitespace(text):
  text = text.strip()
  return text
#Remove Punctuation
def remove_punctuation(text):
  punct = string.punctuation
  punct = punct.replace("'","")
  text = text.translate(str.maketrans("", "",punct))
  return text
#Remove HTML
def remove_html(text):
  html = re.compile(r'<.*?>')
  text = html.sub(r'',text)
  return text
# Removing emojis
def remove_emoji(text):
  emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    "]+",flags=re.UNICODE
  )
  text = emoji_pattern.sub(r'',text)
  return text
#Remove HTML
def remove_html(text):
  html = re.compile(r'<.*?>')
  text = html.sub(r'',text)
  return text
#Remove URLS
def remove_http_links(text):
  text = re.sub('http://\S+|https://\S+','',text)
  return text
#Convert Contractions like you're
def convert_contractions(text):
  text = contractions.fix(text)
  return text
#Remove Stopwords
def remove_stopwords(text):
  text = " ".join([word for word in nltk.tokenize.word_tokenize(text) if word not in stopwords.words('english')])
  return text
# Lemmatization
nlp = spacy.load("en_core_web_sm", disable = ['parser', 'ner'])

def lemmatize(text):
  text = " ".join([token.lemma_ for token in nlp(text)])
  return text
#Remove Non-Alphabetic Characters
def discard_non_alpha(text):
  word_list_non_alpha = [word for word in regexp.tokenize(text) if word.isalpha()]
  text = " ".join(word_list_non_alpha)
  return text

In [8]:
def preprocess_text(text):
  text = text_lower(text)
  text = remove_whitespace(text)
  text = re.sub('\n' , '', text)
  text = re.sub('\[.*?\]', '', text)
  text = remove_http_links(text)
  text = remove_punctuation(text)
  text = remove_html(text)
  text = remove_emoji(text)
  text = convert_contractions(text)
  text = remove_stopwords(text)
  text = discard_non_alpha(text)
  text = lemmatize(text)

  return text

# Importing data

In [4]:
ai=pd.read_csv('/content/drive/MyDrive/nlp data/ai_articles.csv')
ml=pd.read_csv('/content/drive/MyDrive/nlp data/ml_articles.csv')
explainable_ai=pd.read_csv('/content/drive/MyDrive/nlp data/explainable_articles.csv')
deep_learning=pd.read_csv('/content/drive/MyDrive/nlp data/dl_articles.csv')
data_mining=pd.read_csv('/content/drive/MyDrive/nlp data/dm_articles.csv')
gpt_art=pd.read_csv('/content/drive/MyDrive/nlp data/gpt_articles.csv')
nlp=pd.read_csv('/content/drive/MyDrive/nlp data/nlp_articles.csv')
reinforcement_learning=pd.read_csv('/content/drive/MyDrive/nlp data/rl_articles.csv')
transfer_leaning=pd.read_csv('/content/drive/MyDrive/nlp data/transfer_articles.csv')

In [5]:
combined_data = pd.DataFrame()
file_paths=[ai,ml,explainable_ai,deep_learning,data_mining,gpt_art,nlp,reinforcement_learning,transfer_leaning]
# Iterate through the files and combine them
for file in file_paths:
    combined_data = combined_data.append(file, ignore_index=True)

# Save the combined data to a new CSV file
combined_data.to_csv('combined_data.csv', index=False)

<ipython-input-5-e3186fa45b8b>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(file, ignore_index=True)


In [6]:
combined_data

,keyword,url,title,content,urlToImage,author
0,artificial intelligence,https://devops.com/digital-ai-update-extends-s...,Digital.ai Update Extends Scope and Reach of D...,Digital.ai this week made generally available ...,https://devops.com/wp-content/uploads/2022/08/...,Mike Vizard
1,artificial intelligence,https://biztoc.com/x/b03439aa640e0416,The battle between open source and 'sort of' o...,\n \n\n\n \n Opinion At the L...,https://c.biztoc.com/p/b03439aa640e0416/s.webp,theregister.com
2,artificial intelligence,https://www.thestranger.com/slog-am/2023/10/27...,"Slog AM: Washington Clears Rape Kit Backlog, U...",\n \n \n\n \n\n \n\n \n \n...,https://media2.fdncms.com/stranger/imager/u/la...,Nathalie Graham
3,artificial intelligence,https://therealdeal.com/sanfrancisco/2023/10/2...,Leasing Offices Now Takes Longer in San Francisco,It takes longer to close deals for office leas...,https://static.therealdeal.com/wp-content/uplo...,TRD Staff
4,artificial intelligence,https://observer.com/2023/10/craig-newmark-int...,Craig Newmark of Craigslist Has a New Mission:...,\n\t\t\n\t\tAt a cheap card table in a South M...,https://observer.com/wp-content/uploads/sites/...,"Alexandra Tremayne-Pengelly, Alexandra Tremayn..."
...,...,...,...,...,...,...
819,transfer learning,https://www.finextra.com/blogposting/25082/ban...,Banking Technology: Delivering the Next-Gen UX...,\n\t\nAs the usage of physical cash diminishes...,https://www.finextra.com/finextra-images/membe...,Editorial Team
820,transfer learning,https://www.gettingsmart.com/2023/10/24/the-gl...,"The Global Credentialing Landscape: Messy, Mas...",\n\t\t\t\t\t\n\t\n\t\t\n \n \n\t\t\n\t\t\t\t ...,https://www.gettingsmart.com/wp-content/upload...,Guest Author
821,transfer learning,https://www.javacodegeeks.com/2023/10/transfor...,Transforming Data Analysis: The Game-Changing ...,In the rapidly evolving landscape of data anal...,https://www.javacodegeeks.com/wp-content/uploa...,Java Code Geeks
822,transfer learning,https://www.irishtimes.com/ireland/education/2...,My daughter wants to study nursing but I worry...,A well-motivated school leaver can now aspire ...,https://www.irishtimes.com/resizer/EdTNpkWwcfi...,Brian Mooney
